In [1]:
import pandas as pd 
import numpy as np
import itertools

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('/var/www/universityJobDay/universityJobDay/database/answers.db')
#conn = sqlite3.connect('/home/daniel/git/universityJobDay/database/answers.db')

In [4]:
df = pd.read_sql_query('SELECT * from answers', conn)

In [5]:
df.head()

,id,sessionID,f1,f2,f3,f4,f5,f6,date
0,1,2N01KFSNC2RQIZM,1,3,3~0~2,4,1,1,2020-05-20 05:40:56.670853
1,2,EO29HHPJT4CYAO2,0,1,3~1,3,Risiko Datenschutz + Privatsphäre,2,2020-05-20 05:46:35.477319
2,3,2EL8BUZB5J0O873,0,3,6~0,4,"Chancen: Sicherheit, Prävention\nRisiken: der ...",2,2020-05-20 05:53:47.294080
3,4,C1BRP0FGCG6YSHS,0,3,0~1,2,"Chancen: Minimierung von Fehlern\nRisiken: ""En...",2,2020-05-20 06:13:47.718848
4,5,CP5VTIEK8FNC22P,1,3,0~2~1,2,Analyse von vielen unstrukturierten und auf de...,0,2020-05-20 06:24:26.482387


In [6]:
conn.close()

In [7]:
f3 = df["f3"].apply(lambda x: x.split("~"))

In [8]:
f3 = pd.DataFrame([entry + ["-99"] * (3-len(entry)) for entry in f3], columns = ["f3_1", "f3_2", "f3_3"]).astype(int)

In [9]:
f3.replace(-99, np.nan, inplace = True)

In [10]:
date = pd.DataFrame([[entry.split(" ")[0], entry.split(" ")[1]] for entry in df["date"]], columns = ["date", "time"])

In [11]:
df = pd.concat([df.loc[:, [col for col in df.columns if col not in  ["f3", "date"]]], f3, date], axis = 1)

In [12]:
df.head()

,id,sessionID,f1,f2,f4,f5,f6,f3_1,f3_2,f3_3,date,time
0,1,2N01KFSNC2RQIZM,1,3,4,1,1,3,0.0,2.0,2020-05-20,05:40:56.670853
1,2,EO29HHPJT4CYAO2,0,1,3,Risiko Datenschutz + Privatsphäre,2,3,1.0,NaN,2020-05-20,05:46:35.477319
2,3,2EL8BUZB5J0O873,0,3,4,"Chancen: Sicherheit, Prävention\nRisiken: der ...",2,6,0.0,NaN,2020-05-20,05:53:47.294080
3,4,C1BRP0FGCG6YSHS,0,3,2,"Chancen: Minimierung von Fehlern\nRisiken: ""En...",2,0,1.0,NaN,2020-05-20,06:13:47.718848
4,5,CP5VTIEK8FNC22P,1,3,2,Analyse von vielen unstrukturierten und auf de...,0,0,2.0,1.0,2020-05-20,06:24:26.482387


In [13]:
for col in ["f1", "f2", "f4", "f6"]:
    df[col] = df[col].astype(int)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         31 non-null     int64  
 1   sessionID  31 non-null     object 
 2   f1         31 non-null     int64  
 3   f2         31 non-null     int64  
 4   f4         31 non-null     int64  
 5   f5         31 non-null     object 
 6   f6         31 non-null     int64  
 7   f3_1       31 non-null     int64  
 8   f3_2       31 non-null     int64  
 9   f3_3       24 non-null     float64
 10  date       31 non-null     object 
 11  time       31 non-null     object 
dtypes: float64(1), int64(7), object(4)
memory usage: 3.0+ KB


In [15]:
def col_percent_single(data, label_dict):
    prep = data.replace(label_dict).value_counts().dropna()
    calculation = prep.apply(lambda x: int(x/data.shape[0]*100))
    return [calculation.index.to_list(), [int(value) for value in calculation.values], [int(value) for value in prep.values], data.shape[0]]

In [16]:
def col_percent_multi(data,label_dict):
    n = data.shape[0]
    prep = data.replace(label_dict).melt().value.dropna().value_counts()
    calculation = prep.apply(lambda x: int(x/n*100))
    return [calculation.index.to_list(), list(calculation.values), list(prep.values), n]

In [17]:
f1_dict = {
    1: "männlich", 
    0: "weiblich", 
    2: "divers"}
f2_dict = {
    0: "18 bis 20 Jahre", 
    1: "21 bis 25 Jahre", 
    2: "26 bis 30 Jahre", 
    3: "älter als 30 Jahre"}
f3_dict = {
    0: "Wissenschaft", 
    1: "Medizin", 
    2: "Arbeit",
    3: "Privater Alltag", 
    4: "Shopping",
    5: "Öffentlicher Nahverkehr",
    6: "Sicherheit",
    7: "Finanzen",
    8: "Gesundheit & Fitness",
    9: "Anderer Bereich"}
f4_dict = {
    0: "Uni",
    1: "Arbeit",
    2: "Medizin",
    3: "Privater Alltag",
    4: "Shopping",
    5: "Öffentlicher Nahverkehr",
    6: "Sicherheit",
    7: "Finanzen",
    8: "Gesundheit & Fitness"}
f6_dict = {
    0: "benötigen mehr KI",
    1: "AI/KI nur Buzzwort",
    2: "Davon gibts schon genug!"}

In [18]:
singles = [col_percent_single(df["f1"].astype(int), f1_dict), col_percent_single(df["f2"].astype(int), f2_dict), col_percent_single(df["f4"].astype(int), f4_dict), col_percent_single(df["f6"].astype(int), f6_dict)]
singles

[[['männlich', 'weiblich'], [54, 45], [17, 14], 31],
 [['älter als 30 Jahre', '21 bis 25 Jahre', '26 bis 30 Jahre'],
  [74, 12, 12],
  [23, 4, 4],
  31],
 [['Arbeit',
   'Öffentlicher Nahverkehr',
   'Medizin',
   'Privater Alltag',
   'Sicherheit',
   'Gesundheit & Fitness',
   'Shopping',
   'Finanzen',
   'Uni'],
  [38, 12, 12, 9, 6, 6, 6, 3, 3],
  [12, 4, 4, 3, 2, 2, 2, 1, 1],
  31],
 [['benötigen mehr KI', 'AI/KI nur Buzzwort', 'Davon gibts schon genug!'],
  [54, 32, 12],
  [17, 10, 4],
  31]]

In [19]:
isinstance(singles[0][1][0], int)

True

In [20]:
multies = [col_percent_multi(df.loc[:, [col for col in df.columns if "f3" in col]], f3_dict)]

In [21]:
df.loc[:, [col for col in df.columns if "f3" in col]].replace(f3_dict).melt().value.dropna().value_counts()

Wissenschaft               20
Medizin                    17
Arbeit                     16
Öffentlicher Nahverkehr    11
Sicherheit                 10
Privater Alltag             5
Finanzen                    3
Gesundheit & Fitness        2
Shopping                    2
Name: value, dtype: int64

In [84]:
#np.ones(df.shape, dtype=bool)

In [86]:
def col_percent_single(data, label_dict):
    """
    Function to calculate column percentage of a single-answer-type question

    args -
    data: pandas series
    label_dict: dictionary with value labels

    returns: list-like [groupnames, percent, groupsize, total_n]
    """

    prep = data.replace(label_dict).value_counts().dropna()
    calculation = prep.apply(lambda x: int(x/data.shape[0]*100))
    
    return [calculation.index.to_list(), [int(value) for value in calculation.values], ['group n = ' + str(value) for value in prep.values], data.shape[0]]

def col_percent_multi(data, label_dict):
    """
    function to calculate colum percentage of a multi-answer-type-question.

    args -
    data: pandas df
    label_dict: dictionary with value labels

    returns: list-like [groupnames, percent, total_answers_per_option, total_n (of participants)]
    """

    n = data.shape[0]
    prep = data.replace(label_dict).melt().value.dropna().value_counts()
    calculation = prep.apply(lambda x: int(x/n*100))
    
    return [calculation.index.to_list(), [int(value) for value in calculation.values], ['group n = ' + str(value) for value in prep.values], n]


In [98]:
def calculate_splits(question, split):
    """
    function to calculate market research column percentages for filtered groups.

    args - 
    - question: list-like --> [question, label dictionairy]
    - filter: list-like --> [question, label_dictionairy]

    returns -
    list-like --> [labels of items, values, group n, total n, labels of splits]
    """
    prep = pd.crosstab(split[0].replace(split[1]), question[0].replace(question[1]))
    calculation = prep.apply(lambda r: (r/r.sum()*100), axis = 1).astype(int)

    int_converted = []
    for _, row in calculation.iterrows():
        int_converted.append([int(entry) for entry in row])

    int_group_n = ["group n = " + str(entry) for entry in prep.sum(axis = 1)]

    return [calculation.columns.to_list(), int_converted, int_group_n, question[0].shape[0], calculation.index.to_list()]

In [112]:
def calculate_split_multi(question, split):
    """
    function to calculate market research column percentages for filtered groups of multi answer questions.

    args - 
    - question: list-like --> [question Dataframe (e.g. all columns for question), label dictionairy]
    - filter: list-like --> [question, label_dictionairy]

    returns -
    list-like --> [labels of items, values, group n, total n, labels of splits]
    """
    merged = pd.concat([question[0].replace(question[1]), split[0].replace(split[1])], axis = 1)
    splitCol = merged.columns[-1]

    melt = merged.melt(id_vars = splitCol)

    prep = pd.crosstab(melt[splitCol], melt["value"])
    calculation = prep.apply(lambda r: (r/r.sum()*100), axis = 1).astype(int)

    int_converted = []
    for _, row in calculation.iterrows():
        int_converted.append([int(entry) for entry in row])

    int_group_n = ["group n = " + str(entry) for entry in prep.sum(axis = 1)]

    return [calculation.columns.to_list(), int_converted, int_group_n, question[0].shape[0], calculation.index.to_list()]

In [108]:
calculate_splits([df["f4"], f4_dict], [df["f1"], f1_dict])

[['Arbeit',
  'Finanzen',
  'Gesundheit & Fitness',
  'Medizin',
  'Privater Alltag',
  'Shopping',
  'Sicherheit',
  'Uni',
  'Öffentlicher Nahverkehr'],
 [[41, 5, 0, 17, 0, 5, 5, 5, 17], [35, 0, 14, 7, 21, 7, 7, 0, 7]],
 ['group n = 17', 'group n = 14'],
 31,
 ['männlich', 'weiblich']]

In [114]:
calculate_split_multi([df.loc[:, [col for col in df.columns if "f3" in col]], f3_dict], [df["f6"], f1_dict])

[['Arbeit',
  'Finanzen',
  'Gesundheit & Fitness',
  'Medizin',
  'Privater Alltag',
  'Shopping',
  'Sicherheit',
  'Wissenschaft',
  'Öffentlicher Nahverkehr'],
 [[11, 0, 0, 22, 22, 0, 11, 33, 0],
  [25, 0, 3, 17, 3, 0, 17, 17, 14],
  [16, 6, 2, 20, 4, 4, 8, 24, 14]],
 ['group n = 9', 'group n = 28', 'group n = 49'],
 31,
 ['divers', 'männlich', 'weiblich']]